In [1]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("Dates")
using DataFrames, CSV, Dates

Bool2Int(array) = convert(Vector{Int64}, array)
println(Dates.now())

cd(@__DIR__); pwd()

yyyymmdd = DateFormat("yyyymmdd")

@time integrated = CSV.read("C:\\Users\\zzuge\\OneDrive\\문서\\integrated1.csv", DataFrame)
# @time integrated = CSV.read("C:/saved/phase1/integrated.csv", DataFrame)

# is_applied, gender, 회생 관련 불리언으로 변환 및 0,1로 저장
integrated.is_applied = [ismissing(val) ? val : trunc(Int64, isone(val)) for val in integrated.is_applied]
integrated.gender = [ismissing(val) ? val : trunc(Int64, isone(val)) for val in integrated.gender]
integrated.personal_rehabilitation_yn = [ismissing(val) ? val : trunc(Int64, isone(val)) for val in integrated.personal_rehabilitation_yn]
integrated.personal_rehabilitation_complete_yn = [ismissing(val) ? val : trunc(Int64, isone(val)) for val in integrated.personal_rehabilitation_complete_yn]

# user_id, credit_score 정수로 변환
integrated.user_id = [ismissing(val) ? val : trunc(Int64, val) for val in integrated.user_id]
integrated.credit_score = [ismissing(val) ? val : trunc(Int64, val) for val in integrated.credit_score]

# 입사연월을 경력으로 변환
temp = integrated.company_enter_month
temp[ismissing.(temp)] .= 20220801.0
temp = trunc.(Int64, temp)
bit_digit6 = (temp .< 1000000)
temp[bit_digit6] .*= 100
temp[bit_digit6] .+= 1
temp = string.(temp)
temp = Date.(temp, DateFormat("yyyymmdd"))
integrated.company_enter_month = Dates.value.(Date(2022, 8) .- temp)
integrated.company_enter_month[integrated.company_enter_month .< 0] .= 0
rename!(integrated, :company_enter_month => :career)

# # EDA 당시 아래의 상태에서 저장
# CSV.write("../01_data/integrated_preview.csv", integrated[rand(1:nrow(integrated), 100),:], bom = true)
# @time CSV.write("../01_data/integrated.csv", integrated, bom = true)

# ---------------------------

# 분석을 위한 데이터 정제 작업 시작

is_applied = integrated.is_applied
is_goodbank = ((integrated.bank_id .== 4) .+ (integrated.bank_id .== 29))
is_goodprod = ((integrated.product_id .== 24) .+ (integrated.product_id .== 47) .+ (integrated.product_id .== 84))

nl_loan_limit = abs.(log10.(integrated.loan_limit .+ 1) .- 7)
nl_loan_rate  = abs.(integrated.loan_rate .- 13) # 비선형모델 개발 필요
is_monday = iszero.(integrated.weekday) |> Bool2Int
is_weekend = (5 .≤ integrated.weekday) |> Bool2Int
is_after_16 = (integrated.insert_time .> 16) |> Bool2Int

gender = integrated.gender
credit_score = integrated.credit_score
application_n = integrated.application_n
loyalty = integrated.loyalty

log_yearly_income = log10.(integrated.yearly_income .+ 1)
is_earned_income1 = (integrated.income_type .== "EARNEDINCOME") |> Bool2Int
is_earned_income2 = (integrated.income_type .== "EARNEDINCOME2") |> Bool2Int

log_career = log10.(integrated.career .+ 1)
is_job_regular = (integrated.employment_type .== "정규직") |> Bool2Int
is_job_etc     = (integrated.employment_type .== "기타") |> Bool2Int
is_house_own   = (integrated.houseown_type .== "자가") |> Bool2Int
is_house_lent  = (integrated.houseown_type .== "전월세") |> Bool2Int

log_desired_amount = log10.(integrated.desired_amount .+ 1) # 비선형모델 개발

is_purpose_living  = (integrated.purpose .== "생활비") |> Bool2Int
personal_rehabilitation_yn = integrated.personal_rehabilitation_yn
personal_rehabilitation_complete_yn = integrated.personal_rehabilitation_complete_yn
personal_rehabilitation_yn[ismissing.(personal_rehabilitation_yn)] .= 0
personal_rehabilitation_complete_yn[ismissing.(personal_rehabilitation_complete_yn)] .= 0

log_existing_loan_cnt = log10.(integrated.existing_loan_cnt) # 비선형모델 개발
log_existing_loan_amt = log10.(integrated.existing_loan_amt .+ 1) # 비선형모델 개발

nl_age  = abs.(log10.(integrated.age) .- 1.6) # 비선형모델 개발

log_OpenApp = log10.(integrated.OpenApp .+ 1)
log_UseDSRCalc = log10.(integrated.UseDSRCalc .+ 1)

# SingUp은 일단 삭제(조사 중)
# OpenApp, UseDSRCalc 만 사용

application_id = integrated.application_id
product_id = integrated.product_id

new_integrated = DataFrame(
; application_id
, product_id
, is_applied
, credit_score
, application_n
, loyalty
, gender
, log_yearly_income
, log_career
, log_desired_amount
, log_existing_loan_cnt
, log_existing_loan_amt
, log_OpenApp
, log_UseDSRCalc
, nl_loan_limit
, nl_loan_rate
, nl_age
, is_goodbank
, is_goodprod
, is_monday
, is_weekend
, is_after_16
, is_earned_income1
, is_earned_income2
, is_job_regular
, is_job_etc
, is_house_own
, is_house_lent
, is_purpose_living
, personal_rehabilitation_yn
, personal_rehabilitation_complete_yn
)

test = new_integrated[ismissing.(new_integrated.is_applied),:]
train = dropmissing(new_integrated[.!ismissing.(new_integrated.is_applied),:])

CSV.write("C:\\Users\\zzuge\\test_preview.csv", test[rand(1:nrow(test), 100),:], bom = true)
@time CSV.write("C:\\Users\\zzuge\\test.csv", test, bom = true)
@time CSV.write("C:\\Users\\zzuge\\train.csv", train, bom = true)

println(Dates.now())


# --- ▲ v1
foo(x) = cos((x-13)/2)
#plot(0:0.1:20, foo.(0:0.1:20))

nl_loan_rate  = cos.((integrated.loan_rate .- 13)/2) # 비선형모델 개발 필요

new_integrated = DataFrame(
; application_id
, product_id
, is_applied
, credit_score
, application_n
, loyalty
, gender
, log_yearly_income
, log_career
, log_desired_amount
, log_existing_loan_cnt
, log_existing_loan_amt
, log_OpenApp
, log_UseDSRCalc
, nl_loan_limit
, nl_loan_rate
, nl_age
, is_goodbank
, is_goodprod
, is_monday
, is_weekend
, is_after_16
, is_earned_income1
, is_earned_income2
, is_job_regular
, is_job_etc
, is_house_own
, is_house_lent
, is_purpose_living
, personal_rehabilitation_yn
, personal_rehabilitation_complete_yn
)

test = new_integrated[ismissing.(new_integrated.is_applied),:]
train = dropmissing(new_integrated[.!ismissing.(new_integrated.is_applied),:])

CSV.write("C:\\Users\\zzuge\\test_preview.csv", test[rand(1:nrow(test), 100),:], bom = true)
@time CSV.write("C:\\Users\\zzuge\\test.csv", test, bom = true)
@time CSV.write("C:\\Users\\zzuge\\train.csv", train, bom = true)


    Updating registry at `C:\Users\zzuge\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\zzuge\.julia\environments\v1.8\Manifest.toml`


2022-10-08T20:41:26.937
 38.892488 seconds (2.04 M allocations: 5.859 GiB, 3.98% gc time, 16.99% compilation time)
 15.585816 seconds (444.31 M allocations: 11.139 GiB, 15.52% gc time)
 24.048823 seconds (946.41 M allocations: 23.696 GiB, 10.44% gc time, 0.60% compilation time)
2022-10-08T20:43:20.904
 13.115869 seconds (444.31 M allocations: 11.139 GiB, 14.47% gc time)
 23.663231 seconds (945.74 M allocations: 23.664 GiB, 10.24% gc time)


"C:\\Users\\zzuge\\train.csv"